In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.cuda.amp import autocast, GradScaler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CasualSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=False)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=False)

        self.n_head = config.n_head
        self.n_embd = config.n_embd

        self.register_buffer(
            "bias",
            torch.tril(torch.ones(config.block_size, config.block_size))
            .view(1, 1, config.block_size, config.block_size),
        )

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.c_attn(x)

        q, k, v = qkv.split(self.n_embd, dim=2)

        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) * (1.0 / (C // self.n_head) ** 0.5)

        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float("-inf"))
        att = F.softmax(att, dim=-1)

        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        y = self.c_proj(y)
        return y


class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate="tanh")
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x


class Block(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.ln1 = nn.LayerNorm(config.n_embd)
        self.attention = CasualSelfAttention(config)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attention(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 128
    vocab_size: int = 50257
    n_layer: int = 8
    n_head: int = 8
    n_embd: int = 512
    dropout: float = 0.2


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(
            dict(
                token_embedding=nn.Embedding(config.vocab_size, config.n_embd),
                position_embedding=nn.Embedding(config.block_size, config.n_embd),
                h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
                ln_f=nn.LayerNorm(config.n_embd),
            )
        )

        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    def forward(self, idx, targets=None):
        B, T = idx.size()
        token_embeddings = self.transformer.token_embedding(idx)
        position_ids = torch.arange(T, device=idx.device).unsqueeze(0)
        position_embeddings = self.transformer.position_embedding(position_ids)
        x = token_embeddings + position_embeddings
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss


class GPTDataset(torch.utils.data.Dataset):
    def __init__(self, path, block_size):
        self.data = np.memmap(path, dtype=np.uint16, mode="r")
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        chunk = self.data[idx : idx + self.block_size + 1]
        x = torch.tensor(chunk[:-1], dtype=torch.long)
        y = torch.tensor(chunk[1:], dtype=torch.long)
        return x, y



block_size = 128
batch_size = 96
train_steps = 20000
save_path = "/content/drive/MyDrive/gpt8x512.pth"


train_dataset = GPTDataset("/content/drive/MyDrive/bookcorpus_tokens.bin", block_size)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4
)


config = GPTConfig(block_size=block_size)
model = GPT(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
scaler = GradScaler()


step = 0
while step < train_steps:
    for x_batch, y_batch in train_loader:
        if step >= train_steps:
            break
        model.train()
        x_batch, y_batch = x_batch.to(device, non_blocking=True), y_batch.to(device, non_blocking=True)

        optimizer.zero_grad()
        with autocast():
            logits, loss = model(x_batch, y_batch)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if step % 50 == 0:
            print(f"Step {step}, Loss: {loss.item():.4f}")

        if step % 500 == 0 and step > 0:
            torch.save(model.state_dict(), save_path)
            print(f"Model saved to {save_path}")

        step += 1


Mounted at /content/drive


/tmp/ipython-input-1-4043524157.py:153: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipython-input-1-4043524157.py:165: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Step 0, Loss: 11.0159
Step 50, Loss: 6.4785
Step 100, Loss: 5.9868
Step 150, Loss: 5.5971
Step 200, Loss: 5.6113
Step 250, Loss: 5.4783
Step 300, Loss: 5.3896
Step 350, Loss: 5.3517
Step 400, Loss: 5.2054
Step 450, Loss: 5.1116
Step 500, Loss: 5.0215
Model saved to /content/drive/MyDrive/gpt8x512.pth
Step 550, Loss: 5.0513
Step 600, Loss: 4.9194
Step 650, Loss: 4.8346
Step 700, Loss: 4.9784
Step 750, Loss: 4.8763
Step 800, Loss: 4.8115
Step 850, Loss: 4.7562
Step 900, Loss: 4.7052
Step 950, Loss: 4.7728
Step 1000, Loss: 4.7358
Model saved to /content/drive/MyDrive/gpt8x512.pth
Step 1050, Loss: 4.6686
Step 1100, Loss: 4.6747
Step 1150, Loss: 4.6122
Step 1200, Loss: 4.6696
Step 1250, Loss: 4.5487
Step 1300, Loss: 4.5601
Step 1350, Loss: 4.6381
Step 1400, Loss: 4.5630
Step 1450, Loss: 4.5715
Step 1500, Loss: 4.5216
Model saved to /content/drive/MyDrive/gpt8x512.pth
Step 1550, Loss: 4.3642
Step 1600, Loss: 4.4098
Step 1650, Loss: 4.3822
Step 1700, Loss: 4.4998
Step 1750, Loss: 4.4212
Step 